In [1]:
import csv
import os
import gc
import numpy as np
import tensorflow as tf
from random import randint
from tensorflow import keras
from keras.utils import np_utils, plot_model, to_categorical
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Input, LSTM, Dense, RepeatVector, TimeDistributed, concatenate

In [2]:
# returns train, inference_encoder and inference_decoder models
def define_models(dim_p, n_units):
	# define training encoder
	enc_in_o = Input(shape=(None, 1))
	enc_in_q = Input(shape=(None, 1))
	enc_in_p = Input(shape=(None, dim_p))
	encoder_inputs = concatenate([enc_in_o, enc_in_q, enc_in_p])
	encoder = LSTM(n_units, return_state=True)
	encoder_outputs, state_h, state_c = encoder(encoder_inputs)
	encoder_states = [state_h, state_c]
	
	# define training decoder 1
	dec_in_o = Input(shape=(None, 1))
	dec_in_q = Input(shape=(None, 1))
	dec_in_p = Input(shape=(None, dim_p))
	decoder_inputs = concatenate([dec_in_o, dec_in_q, dec_in_p])
	decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
	decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
	dec_dense_o = Dense(1, activation='sigmoid', name='tr_out_o')
	dec_dense_q = Dense(1, activation='sigmoid', name='tr_out_q')
	dec_dense_p = Dense(dim_p, activation='softmax', name='tr_out_p')
	#out_o = Dense(1, activation='relu', name='tr_out_o')(decoder_outputs)#act relu
	#out_q = Dense(1, activation='sigmoid', name='tr_out_q')(decoder_outputs)
	#out_p = Dense(dim_p, activation='softmax', name='tr_out_p')(decoder_outputs)
	out_o = dec_dense_o(decoder_outputs)
	out_q = dec_dense_q(decoder_outputs)
	out_p = dec_dense_p(decoder_outputs)

	# define training decoder 2
	dec_in_o_2 = Input(shape=(None, 1))
	dec_in_q_2 = Input(shape=(None, 1))
	dec_in_p_2 = Input(shape=(None, dim_p))
	decoder_inputs_2 = concatenate([dec_in_o_2, dec_in_q_2, dec_in_p_2])
	decoder_lstm_2 = LSTM(n_units, return_sequences=True, return_state=True)
	decoder_outputs_2, _, _ = decoder_lstm_2(decoder_inputs_2, initial_state=encoder_states)
	dec_dense_o_2 = Dense(1, activation='relu', name='tr_out_o_2')
	dec_dense_q_2 = Dense(1, activation='relu', name='tr_out_q_2')
	dec_dense_p_2 = Dense(dim_p, activation='softmax', name='tr_out_p_2')
	#out_o = Dense(1, activation='relu', name='tr_out_o')(decoder_outputs)#act relu
	#out_q = Dense(1, activation='sigmoid', name='tr_out_q')(decoder_outputs)
	#out_p = Dense(dim_p, activation='softmax', name='tr_out_p')(decoder_outputs)
	out_o_2 = dec_dense_o_2(decoder_outputs_2)
	out_q_2 = dec_dense_q_2(decoder_outputs_2)
	out_p_2 = dec_dense_p_2(decoder_outputs_2)
	
	model = Model([enc_in_o, enc_in_q, enc_in_p, dec_in_o, dec_in_q, dec_in_p], [out_o, out_q, out_p])
	model_2 = Model([enc_in_o, enc_in_q, enc_in_p, dec_in_o_2, dec_in_q_2, dec_in_p_2], [out_o_2, out_q_2, out_p_2])
	
	# define inference encoder
	encoder_model = Model([enc_in_o, enc_in_q, enc_in_p], encoder_states)
	
	# define inference decoder 1
	decoder_state_input_h = Input(shape=(n_units,))
	decoder_state_input_c = Input(shape=(n_units,))
	decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
	decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
	decoder_states = [state_h, state_c]
	#out_o = TimeDistributed(Dense(1, activation='relu'))(decoder_outputs)#act relu
	#out_q = TimeDistributed(Dense(1, activation='sigmoid'))(decoder_outputs)
	#out_p = TimeDistributed(Dense(dim_p, activation='softmax'))(decoder_outputs)
	out_o = dec_dense_o(decoder_outputs)
	out_q = dec_dense_q(decoder_outputs)
	out_p = dec_dense_p(decoder_outputs)

	# define inference decoder 2
	decoder_state_input_h_2 = Input(shape=(n_units,))
	decoder_state_input_c_2 = Input(shape=(n_units,))
	decoder_states_inputs_2 = [decoder_state_input_h, decoder_state_input_c_2]
	decoder_outputs_2, state_h_2, state_c_2 = decoder_lstm_2(decoder_inputs_2, initial_state=decoder_states_inputs_2)
	decoder_states_2 = [state_h_2, state_c_2]
	#out_o = TimeDistributed(Dense(1, activation='relu'))(decoder_outputs)#act relu
	#out_q = TimeDistributed(Dense(1, activation='sigmoid'))(decoder_outputs)
	#out_p = TimeDistributed(Dense(dim_p, activation='softmax'))(decoder_outputs)
	out_o_2 = dec_dense_o_2(decoder_outputs_2)
	out_q_2 = dec_dense_q_2(decoder_outputs_2)
	out_p_2 = dec_dense_p_2(decoder_outputs_2)

	decoder_model = Model([dec_in_o, dec_in_q, dec_in_p] + decoder_states_inputs, [out_o, out_q, out_p] + decoder_states)
	decoder_model_2 = Model([dec_in_o_2, dec_in_q_2, dec_in_p_2] + decoder_states_inputs_2, [out_o_2, out_q_2, out_p_2] + decoder_states_2)

	# return all models
	return model, model_2, encoder_model, decoder_model, decoder_model_2


# generate target given source sequence
def predict_sequence(infenc, infdec, src_o, src_q, src_p, n_steps, cardinality):
	# encode
	state = infenc.predict([src_o, src_q, src_p])
	# start of sequence input
	target_o = np.array([-1]).reshape(1, 1, 1)
	target_q = np.array([-1]).reshape(1, 1, 1)
	#target_p = 0
	target_p = np.array([0.0 for _ in range(cardinality)]).reshape(1, 1, cardinality)
	# collect predictions
	output = list()
	for t in range(n_steps):
		# predict next char
		#print(target_o.shape)
		#print(target_q.shape)
		#print(target_p.shape)
		#print(state[0].shape)
		o, q, p, h, c = infdec.predict([target_o, target_q, target_p] + state)
		#print(a)
		# store prediction
		output.append(o[0,0,:])
		output.append(q[0,0,:])
		output.append(p[0,0,:])
		# update state
		state = [h, c]
		# update target sequence
		target_o = o
		target_q = q
		target_p = p
	return np.array(output)

# decode a one hot encoded string
def one_hot_decode(encoded_seq):
	return [np.argmax(vector) for vector in encoded_seq]

#create list with window length sequences of list a data
def rolling_window(a, window):
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

#generate train inputs and outputs while one hot encoding pitch and padding for seq2seq
def generatorex(features1, features2, features3, seq_length, batch_size):
    # Create empty arrays to contain batch of features and labels# 
    batch_features1 = np.zeros((batch_size, seq_length, 1))
    batch_features2 = np.zeros((batch_size, seq_length, 1))
    batch_features3 = np.zeros((batch_size, seq_length, 128))
    batch_feat_pad1 = np.zeros((batch_size, seq_length, 1))
    batch_feat_pad2 = np.zeros((batch_size, seq_length, 1))
    batch_feat_pad3 = np.zeros((batch_size, seq_length, 128))
    i = 0
    while True:
        for b in range(batch_size):
            batch_features1[b] = features1[i]
            batch_features2[b] = features2[i]
            batch_features3[b] = to_categorical(features3[i], num_classes=128)
            batch_feat_pad1[b] = np.append([-1], features1[i][:-1]).reshape(seq_length, 1)
            batch_feat_pad2[b] = np.append([-1], features2[i][:-1]).reshape(seq_length, 1)
            batch_feat_pad3[b] = to_categorical(np.append([0], features3[i][:-1]).reshape(seq_length, 1), num_classes=128)
            i += 1
            if (i == len(features1)):
                i=0
        #print(batch_features, batch_labels)
        yield [batch_features1, batch_features2, batch_features3, batch_feat_pad1, batch_feat_pad2, batch_feat_pad3], [batch_features1, batch_features2, batch_features3]

In [3]:
#load data
stream_list = []
stream_list_2 = []

for path, subdirectories, files in os.walk('/kaggle/input/data-rock/kag'):
    for name in files:
        with open(os.path.join(path, name), 'r') as f: 
            reader = csv.reader(f)
            sub_list = [list(map(float,rec)) for rec in csv.reader(f, delimiter=',')]
            stream_list = stream_list + sub_list
            
for path, subdirectories, files in os.walk('/kaggle/input/data-jazz'):
    for name in files:
        with open(os.path.join(path, name), 'r') as f: 
            reader = csv.reader(f)
            sub_list = [list(map(float,rec)) for rec in csv.reader(f, delimiter=',')]
            stream_list_2 = stream_list_2 + sub_list

In [4]:
#create seperate data structures for each variable (offset, quarterlength, pitch)
#normalise offset and quarterlength
offs = []
qlngth = []
ptch = []

offs_2 = []
qlngth_2 = []
ptch_2 = []

offsb = max(element[0] for element in stream_list if element[0]<=600.0)
qlngthb = max(element[1] for element in stream_list if element[1]<=50.0)
#ptchb = 127.0
offsb_2 = max(element[0] for element in stream_list_2 if element[0]<=600.0)
qlngthb_2 = max(element[1] for element in stream_list_2 if element[1]<=50.0)

for row in stream_list:
    if (row[0] <= 600.0 and row[1] <= 50.0):
        offs.append(row[0]/offsb)
        qlngth.append(row[1]/qlngthb)
        ptch.append(row[2])
        
for row in stream_list_2:
    if (row[0] <= 600.0 and row[1] <= 50.0):
        offs_2.append(row[0]/offsb_2)
        qlngth_2.append(row[1]/qlngthb_2)
        ptch_2.append(row[2])

In [5]:
#divide the sets in sequences of specific length 
dtlngth=[len(offs), len(offs_2)]
seq_length = 4#100 groups of 3

dataX1_o = rolling_window(np.asarray(offs), seq_length)
dataX1_q = rolling_window(np.asarray(qlngth), seq_length)
dataX1_p = rolling_window(np.asarray(ptch), seq_length)

dataX1_o_2 = rolling_window(np.asarray(offs_2), seq_length)
dataX1_q_2 = rolling_window(np.asarray(qlngth_2), seq_length)
dataX1_p_2 = rolling_window(np.asarray(ptch_2), seq_length)

n_patterns = [len(dataX1_p), len(dataX1_p_2)]
print ("Total Patterns: ", n_patterns)

Total Patterns:  [465352, 322353]


In [6]:
#reshape inputs to be [samples, time steps, features]
dataX1_o = np.reshape(dataX1_o, (dtlngth[0] - seq_length + 1, seq_length, 1))
dataX1_q = np.reshape(dataX1_q, (dtlngth[0] - seq_length + 1, seq_length, 1))
dataX1_p = np.reshape(dataX1_p, (dtlngth[0] - seq_length + 1, seq_length, 1))

dataX1_o_2 = np.reshape(dataX1_o_2, (dtlngth[1] - seq_length + 1, seq_length, 1))
dataX1_q_2 = np.reshape(dataX1_q_2, (dtlngth[1] - seq_length + 1, seq_length, 1))
dataX1_p_2 = np.reshape(dataX1_p_2, (dtlngth[1] - seq_length + 1, seq_length, 1))

In [7]:
#divide data in train and validation sets
split_i = [n_patterns[0]*10 // 100, n_patterns[1]*10 // 100]

dataX1_o_v = dataX1_o[-split_i[0]:]
dataX1_o = dataX1_o[:-split_i[0]]

dataX1_q_v = dataX1_q[-split_i[0]:]
dataX1_q = dataX1_q[:-split_i[0]]

dataX1_p_v = dataX1_p[-split_i[0]:]
dataX1_p = dataX1_p[:-split_i[0]]

dataX1_o_v_2 = dataX1_o_2[-split_i[1]:]
dataX1_o_2 = dataX1_o_2[:-split_i[1]]

dataX1_q_v_2 = dataX1_q_2[-split_i[1]:]
dataX1_q_2 = dataX1_q_2[:-split_i[1]]

dataX1_p_v_2 = dataX1_p_2[-split_i[1]:]
dataX1_p_2 = dataX1_p_2[:-split_i[1]]

print ("Validation Patterns: ", split_i)

Validation Patterns:  [46535, 32235]


In [8]:
# configure problem
n_features = 127+1
n_steps_out = seq_length
# define models
train, train_2, infenc, infdec, infdec_2 = define_models(n_features, 256)
train.compile(optimizer='adam', loss={'tr_out_o': 'mse', 'tr_out_q': 'mse', 'tr_out_p': 'categorical_crossentropy'},
 metrics={'tr_out_o': 'mean_squared_error', 'tr_out_q': 'mean_squared_error', 'tr_out_p': 'accuracy'})
train_2.compile(optimizer='adam', loss={'tr_out_o_2': 'mse', 'tr_out_q_2': 'mse', 'tr_out_p_2': 'categorical_crossentropy'},
 metrics={'tr_out_o_2': 'mean_squared_error', 'tr_out_q_2': 'mean_squared_error', 'tr_out_p_2': 'accuracy'})

In [9]:
# train the two models with alternating epochs
epochs_c = 20
for i in range(epochs_c):
    print ("Epoch", str(i+1)+"_a")
    train.fit(generatorex(dataX1_o, dataX1_q, dataX1_p, seq_length, batch_size=5400), steps_per_epoch= (dtlngth[0]-split_i[0]) // 5400)
    print ("Epoch", str(i+1)+"_b")
    train_2.fit(generatorex(dataX1_o_2, dataX1_q_2, dataX1_p_2, seq_length, batch_size=5400), steps_per_epoch= (dtlngth[1]-split_i[1]) // 5400)

Epoch 1_a
77/77 [==============================] - 98s 1s/step - loss: 4.1182 - tr_out_o_loss: 0.0400 - tr_out_q_loss: 0.0246 - tr_out_p_loss: 4.0536 - tr_out_o_mean_squared_error: 0.0400 - tr_out_q_mean_squared_error: 0.0246 - tr_out_p_accuracy: 0.0741
Epoch 1_b
53/53 [==============================] - 64s 1s/step - loss: 4.0246 - tr_out_o_2_loss: 0.0510 - tr_out_q_2_loss: 0.0011 - tr_out_p_2_loss: 3.9725 - tr_out_o_2_mean_squared_error: 0.0510 - tr_out_q_2_mean_squared_error: 0.0011 - tr_out_p_2_accuracy: 0.0783
Epoch 2_a
77/77 [==============================] - 93s 1s/step - loss: 2.8978 - tr_out_o_loss: 0.0125 - tr_out_q_loss: 0.0018 - tr_out_p_loss: 2.8834 - tr_out_o_mean_squared_error: 0.0125 - tr_out_q_mean_squared_error: 0.0018 - tr_out_p_accuracy: 0.2627
Epoch 2_b
53/53 [==============================] - 64s 1s/step - loss: 3.0246 - tr_out_o_2_loss: 0.0112 - tr_out_q_2_loss: 0.0011 - tr_out_p_2_loss: 3.0123 - tr_out_o_2_mean_squared_error: 0.0112 - tr_out_q_2_mean_squared_erro

In [10]:
# spot check some examples
for _ in range(10):
    i = randint(1, split_i[0])
    X1_o = np.reshape(dataX1_o_v[i], (1, seq_length, 1))
    X1_q = np.reshape(dataX1_q_v[i], (1, seq_length, 1))
    X1_p = np.reshape(to_categorical(dataX1_p_v[i], num_classes=n_features), (1, seq_length, n_features))
    target = predict_sequence(infenc, infdec, X1_o, X1_q, X1_p, n_steps_out, n_features)
    for j in range(seq_length):
        print('X_o=%s, y_o=%s, X_q=%s, y_q=%s, X_p=%s, y_p=%s' % (dataX1_o_v[i][j]*offsb, target[3*j]*offsb,
                                                                  dataX1_q_v[i][j]*qlngthb, target[3*j+1]*qlngthb,
                                                                  dataX1_p_v[i][j], one_hot_decode([target[3*j+2]])))
    print()

X_o=[57.5], y_o=[72.76602], X_q=[0.5], y_q=[1.2433934], X_p=[47.], y_p=[47]
X_o=[57.5], y_o=[69.80513], X_q=[0.5], y_q=[0.64134264], X_p=[56.], y_p=[56]
X_o=[57.5], y_o=[67.79031], X_q=[0.5], y_q=[0.51584244], X_p=[59.], y_p=[59]
X_o=[57.5], y_o=[70.969376], X_q=[0.5], y_q=[0.6882849], X_p=[64.], y_p=[64]

X_o=[216.], y_o=[198.64061], X_q=[0.33333333], y_q=[0.92132235], X_p=[68.], y_p=[68]
X_o=[216.], y_o=[182.95241], X_q=[0.5], y_q=[0.7246442], X_p=[66.], y_p=[66]
X_o=[216.5], y_o=[144.06898], X_q=[0.25], y_q=[0.6058459], X_p=[40.], y_p=[40]
X_o=[216.5], y_o=[152.08998], X_q=[0.33333333], y_q=[0.9433565], X_p=[64.], y_p=[64]

X_o=[485.], y_o=[473.0829], X_q=[0.75], y_q=[1.1851158], X_p=[85.], y_p=[85]
X_o=[485.5], y_o=[467.45575], X_q=[0.25], y_q=[0.36616945], X_p=[33.], y_p=[33]
X_o=[485.5], y_o=[483.50064], X_q=[0.5], y_q=[0.32338858], X_p=[68.], y_p=[68]
X_o=[485.5], y_o=[475.38385], X_q=[0.33333333], y_q=[0.6211138], X_p=[69.], y_p=[69]

X_o=[194.], y_o=[181.92165], X_q=[0.5], y_q

In [11]:
# spot check some examples
for _ in range(10):
    i = randint(1, split_i[1])
    X1_o = np.reshape(dataX1_o_v_2[i], (1, seq_length, 1))
    X1_q = np.reshape(dataX1_q_v_2[i], (1, seq_length, 1))
    X1_p = np.reshape(to_categorical(dataX1_p_v_2[i], num_classes=n_features), (1, seq_length, n_features))
    target = predict_sequence(infenc, infdec_2, X1_o, X1_q, X1_p, n_steps_out, n_features)
    for j in range(seq_length):
        print('X_o=%s, y_o=%s, X_q=%s, y_q=%s, X_p=%s, y_p=%s' % (dataX1_o_v_2[i][j]*offsb, target[3*j]*offsb,
                                                                  dataX1_q_v_2[i][j]*qlngthb, target[3*j+1]*qlngthb,
                                                                  dataX1_p_v_2[i][j], one_hot_decode([target[3*j+2]])))
    print()

X_o=[363.5], y_o=[307.48157], X_q=[0.5], y_q=[0.], X_p=[56.], y_p=[56]
X_o=[363.5], y_o=[351.09433], X_q=[0.5], y_q=[0.], X_p=[60.], y_p=[60]
X_o=[364.], y_o=[329.7966], X_q=[0.5], y_q=[0.], X_p=[41.], y_p=[41]
X_o=[364.], y_o=[351.03632], X_q=[0.5], y_q=[0.], X_p=[41.], y_p=[41]

X_o=[536.], y_o=[507.86002], X_q=[1.75], y_q=[0.], X_p=[66.], y_p=[66]
X_o=[536.], y_o=[480.31042], X_q=[8.], y_q=[0.], X_p=[35.], y_p=[35]
X_o=[536.], y_o=[505.43683], X_q=[1.75], y_q=[0.], X_p=[66.], y_p=[66]
X_o=[536.], y_o=[515.209], X_q=[1.75], y_q=[0.], X_p=[66.], y_p=[66]

X_o=[85.5], y_o=[75.58356], X_q=[0.25], y_q=[0.], X_p=[73.], y_p=[73]
X_o=[85.5], y_o=[102.52269], X_q=[0.25], y_q=[0.], X_p=[76.], y_p=[76]
X_o=[85.5], y_o=[91.60635], X_q=[0.25], y_q=[0.], X_p=[85.], y_p=[85]
X_o=[85.5], y_o=[81.90464], X_q=[0.25], y_q=[0.], X_p=[88.], y_p=[88]

X_o=[44.], y_o=[0.], X_q=[8.], y_q=[0.], X_p=[64.], y_p=[64]
X_o=[44.], y_o=[58.536938], X_q=[8.], y_q=[0.], X_p=[68.], y_p=[68]
X_o=[44.], y_o=[37.72735],